# Importing Libraries

In [22]:
import pandas as pd
import numpy as np
import sys 

import warnings
warnings.filterwarnings('ignore')

# Getting data from tubedata.csv

In [23]:
df = pd.read_csv('tubedata.csv', skipinitialspace= True, header= None)
#[StartingStation], [EndingStation], [TubeLine], [AverageTimeTaken], [MainZone], [SecondaryZone]


# Converting data to dictionary

In [26]:
tube_dict = dict() #creating empty dict to store tube data

start_stations = df[0] #getting values of all start stations 
end_stations = df[1] #getting values of all end stations
times = df[3]  #getting values of time taken between stations
main_zones = df[4]  #getting values of all primary stations
sec_zones = df[5] #getting values of all secondary stations

#adding one direction
for i in range(df.shape[0]): #iterating over lenth of dataframe
    start_node = start_stations[i] #start station
    end_node = end_stations[i] #end station
    time_taken = times[i] #time taken
    main_zone =  main_zones[i] #primary zone 
    sec_zone = sec_zones[i] #secondary zone
    
    if start_node not in tube_dict: #checking if start station has a value in dictionary
        #if start node is not present then add value to start node in dictionary
        tube_dict[start_node] = {end_node:[time_taken,main_zone,sec_zone]} 
        
        
    else:
        #if start node is present then just update with new end nodes
        tube_dict[start_node].update({end_node:[time_taken,main_zone,sec_zone]})
        
#adding reverse direction
for i in range(df.shape[0]): #iterating over lenth of dataframe
    start_node = end_stations[i] #start station
    end_node = start_stations[i] #end station
    time_taken = times[i] #time taken
    main_zone =  main_zones[i] #primary zone 
    sec_zone = sec_zones[i] #secondary zone
    
    if start_node not in tube_dict: #checking if start station has a value in dictionary
        #if start node is not present then add value to start node in dictionary
        tube_dict[start_node] = {end_node:[time_taken,main_zone,sec_zone]} 
    else:
        #if start node is present then just update with new end nodes
        tube_dict[start_node].update({end_node:[time_taken,main_zone,sec_zone]})

print(len(tube_dict)) #len of created tube dict

print(tube_dict) #print tube dict

271
{'Harrow & Wealdstone': {'Kenton': [3, '5', '0']}, 'Kenton': {'South Kenton': [2, '4', '0'], 'Harrow & Wealdstone': [3, '5', '0']}, 'South Kenton': {'North Wembley': [2, '4', '0'], 'Kenton': [2, '4', '0']}, 'North Wembley': {'Wembley Central': [2, '4', '0'], 'South Kenton': [2, '4', '0']}, 'Wembley Central': {'Stonebridge Park': [3, '4', '0'], 'North Wembley': [2, '4', '0']}, 'Stonebridge Park': {'Harlesden': [2, '3', '0'], 'Wembley Central': [3, '4', '0']}, 'Harlesden': {'Willesden Junction': [2, '3', '0'], 'Stonebridge Park': [2, '3', '0']}, 'Willesden Junction': {'Kensal Green': [3, '3', '0'], 'Harlesden': [2, '3', '0']}, 'Kensal Green': {"Queen's Park": [3, '2', '0'], 'Willesden Junction': [3, '3', '0']}, "Queen's Park": {'Kilburn Park': [2, '2', '0'], 'Kensal Green': [3, '2', '0']}, 'Kilburn Park': {'Maida Vale': [2, '2', '0'], "Queen's Park": [2, '2', '0']}, 'Maida Vale': {'Warwick Avenue': [1, '2', '0'], 'Kilburn Park': [2, '2', '0']}, 'Warwick Avenue': {'Paddington': [2, '2

In [4]:
#path printing helper function
def print_path(path, cost, nodes):
    print('Shortest path : ',end='')
    try:
        print(path[0],end='')
        for i in range(1,len(path)):
            print('->',end='')
            print(path[i],end='')
        print(f'\nPath cost : {cost}')
        print(f'Nodes Expanded : {nodes}')
    except:
        print('Path not found') 

# Agenda-based Search

## DFS

In [5]:

def depth_first_search(graph, start, end, cost, visited, node_expanded, limit):
    """
    graph - tube dictionary
    start - start station
    end - final destination
    cost - cost to reach station
    visitied - list to store data of visited station
    node_expanded - track of child explored during search
    """
    if limit == 0:
        return [], 0, 0
    if start == end : #check if destination is reached
        return [start], cost, node_expanded 
    
    next_already_visited = visited.copy() #created temp copy of visited codes
    next_already_visited.append(start) # Add current station to already_visited
    
    if start in graph: #check if start station is in dictionary
        paths = [(node, val[0]) for node, val in graph[start].items()] #getting all child nodes of start sation
        paths = paths[::-1] #getting last element in the start node neighbours
        for i in range(len(paths)): #iterationg out length of neighbours
            if paths[i][0] not in visited: #check if neighbour station not in visited node
                #perform depth first search recursively
                visited_nodes, cost, node_expanded = depth_first_search(graph, paths[i][0], end, cost + paths[i][1], next_already_visited, node_expanded+1, limit-1)
                if visited_nodes != []:#check if function does not return null value
                    return [start] + visited_nodes, cost, node_expanded #return shortest path
    
    return [], cost, node_expanded  #if graph start station has no neighbours then return null values

In [6]:
path, cost, nodes = depth_first_search(tube_dict, 'Euston', 'Victoria', 0, [], 0, 30)
print_path(path, cost, nodes)

Shortest path : Euston->King's Cross St. Pancras->Russell Square->Holborn->Tottenham Court Road->Goodge Street->Warren Street->Oxford Circus->Bond Street->Baker Street->Edgware Road->Paddington->Royal Oak->Westbourne Park->Ladbroke Grove->Latimer Road->Shepherd's Bush->Goldhawk Road->Hammersmith->Barons Court->Earls' Court->High Street Kensington->Gloucester Road->South Kensington->Knightsbridge->Hyde Park Corner->Green Park->Victoria
Path cost : 2
Nodes Expanded : 1


## BFS

In [7]:
def breadth_first_search(graph, start, end):
    """
    graph - tube dictionary
    start - start station
    end - final destination
    """
    visited = [start] #array to store visited nodes and adding starting station in visited node
    total_paths = [[start]] #array to store all possible paths
    total_cost = [[0]] #array to store all possible cost 
    node_expanded = 0 #variable to store value of expanded nodes 
    
    while len(total_paths)!= 0: #iterating untill all possible paths are not explored
        
        new_total_paths = list() #array to store new paths from root station
        new_total_costs = list() #array to store new cost from root station
        
        for path,cost in zip(total_paths,total_cost): #pasing over new paths and cost
            if path[-1] in graph: #check if start station has neighbours station present
                for node, val in graph[path[-1]].items():  #getting data of neighbour station
                    if node == end: #check if end is reached
                        return path + [end], sum(cost) + val[0], node_expanded +1 #return shortest path, cost and nodes expanded
                    
                    if node not in visited: #check if neighbour has not been explored
                        #continue the search
                        node_expanded += 1 #increment node expanded
                        visited.append(node) #add current node in visited
                        
                        #update new paths and costs
                        new_total_paths = new_total_paths + [path + [node]] 
                        new_total_costs = new_total_costs + [cost + [val[0]]]
        
        #update total path and cost explored during each graph level
        total_paths = new_total_paths
        total_cost = new_total_costs

    return [],0, -1  #if end station is not reached then return null values 

In [8]:
path, cost, nodes = breadth_first_search(tube_dict, 'Euston', 'Victoria')
print_path(path, cost, nodes)

Shortest path : Euston->Warren Street->Oxford Circus->Green Park->Victoria
Path cost : 7
Nodes Expanded : 30


## UCS

In [20]:
def pop_frontier(frontier):
    if len(frontier) == 0:
        return None
    
    min_val = sys.maxsize
    max_values = list()
    for key, path in frontier:
        if key == min_val:
            max_values.append(path)
        elif key < min_val:
            min_val = key
            max_values.clear()
            max_values.append(path)

    max_values = sorted(max_values, key=lambda x: x[-1])
    # max_values.sort()
    desired_value = max_values[0]
    frontier.remove((min_val, max_values[0]))
    return min_val, desired_value


def get_frontier_params_new(node, frontier):
    for i in range(len(frontier)):
        curr_tuple = frontier[i]
        cost, path = curr_tuple
        if path[-1] == node:
            return True, i, cost, path

    return False, None, None, None


def uniform_cost_search(graph, start, end):   
    """
    graph - tube dictionary
    start - start station
    end - final destination
    """
    path = list()    #array to store path data
    explored_nodes = list() #array to store explored nodes data   
    
    if start == end: #check if starting station is same as end station   
        return path, 0, len(explored_nodes) #if so return null values    
    
    path.append(start)  #add starting sation to path data
    
    path_cost = 0  
    
    frontier = [(path_cost, path)] #priority queue contianing cost and path
    
    while len(frontier) > 0: #check is queue is empty
        
        path_cost_till_now, path_till_now = pop_frontier(frontier)   #get node with min cost
        current_node = path_till_now[-1]    
        explored_nodes.append(current_node)    #append min cost node to explored node array
        
        if current_node in graph:
            if current_node == end:    #check if destination is reached 
                return path_till_now, path_cost_till_now, len(explored_nodes)  #return shortest path , cost and explored nodes   
    
            """
            Sorting paths based on cost 
            """
        
            nodes = []
            vals = []
            for node, val in graph[current_node].items():
                nodes.append(node)
                vals.append(val[0])
            sorted_node = [y for _,y in sorted(zip(vals,nodes),key=lambda x: x[0])]
            sorted_val = sorted(vals)
    
            for neighbour, cost in zip(sorted_node,sorted_val):  #exploring neighbours of start node  
                path_to_neighbour = path_till_now.copy()    
                path_to_neighbour.append(neighbour)    
    
                neighbour_cost = cost + path_cost_till_now   #calculating total cost from start till now 
                new_element = (neighbour_cost, path_to_neighbour)    
    
                is_there, indexx, neighbour_old_cost, _ = get_frontier_params_new(neighbour, frontier)    
    
                if (neighbour not in explored_nodes) and not is_there:    #check if current neighbour not been explored
                    frontier.append(new_element)    
                
                elif is_there:    
                    #updating min cost of nodes 
                    if neighbour_old_cost > neighbour_cost:    
                        frontier.pop(indexx)    
                        frontier.append(new_element)    
    
    return [], path_cost, -1  

In [21]:
path, cost, nodes = uniform_cost_search(tube_dict, 'Euston', 'Victoria')
print_path(path, cost, nodes)

Shortest path : Euston->Warren Street->Oxford Circus->Green Park->Victoria
Path cost : 7
Nodes Expanded : 35


Routes to include in your comparison:
• Canada Water to Stratford
• New Cross Gate to Stepney Green
• Ealing Broadway to South Kensington
• Baker Street to Wembley Park

In [25]:
print("Route 1 Canada Water to Stratford")
print("Using Depth First Search")
path1, cost1, nodes1 = depth_first_search(tube_dict, 'Canada Water', 'Stratford', 0, [], 0, 30)
print_path(path1, cost1, nodes1)
print('---------------------------------')
print("Using Breadth First Search")
path2, cost2, nodes2 = breadth_first_search(tube_dict, 'Canada Water', 'Stratford')
print_path(path2, cost2, nodes2)
print('---------------------------------')
print("Using Uniform cost Search")
path3, cost3, nodes3 = uniform_cost_search(tube_dict, 'Canada Water', 'Stratford')
print_path(path3, cost3, nodes3)

Route 1 Canada Water to Stratford
Using Depth First Search
Shortest path : Canada Water->Bermondsey->London Bridge->Bank/Monument->Moorgate->Liverpool Street->Aldgate East->Whitechapel->Stepney Green->Mile End->Bow Road->Bromley-by-Bow->West Ham->Stratford
Path cost : 44
Nodes Expanded : 19
---------------------------------
Using Breadth First Search
Shortest path : Canada Water->Canary Wharf->North Greenwich->Canning Town->West Ham->Stratford
Path cost : 15
Nodes Expanded : 25
---------------------------------
Using Uniform cost Search
Shortest path : Canada Water->Rotherhithe->Wapping->Shadwell->Whitechapel->Stepney Green->Mile End->Stratford
Path cost : 14
Nodes Expanded : 55


In [26]:
print("Route 2 New Cross Gate to Stepney Green")
print("Using Depth First Search")
path1, cost1, nodes1 = depth_first_search(tube_dict, 'New Cross Gate', 'Stepney Green', 0, [], 0, 30)
print_path(path1, cost1, nodes1)
print('---------------------------------')
print("Using Breadth First Search")
path2, cost2, nodes2 = breadth_first_search(tube_dict, 'New Cross Gate', 'Stepney Green')
print_path(path2, cost2, nodes2)
print('---------------------------------')
print("Using Uniform cost Search")
path3, cost3, nodes3 = uniform_cost_search(tube_dict, 'New Cross Gate', 'Stepney Green')
print_path(path3, cost3, nodes3)

Route 2 New Cross Gate to Stepney Green
Using Depth First Search
Shortest path : New Cross Gate->Surrey Quays->Canada Water->Bermondsey->London Bridge->Bank/Monument->Moorgate->Liverpool Street->Aldgate East->Whitechapel->Stepney Green
Path cost : 23
Nodes Expanded : 10
---------------------------------
Using Breadth First Search
Shortest path : New Cross Gate->Surrey Quays->Canada Water->Rotherhithe->Wapping->Shadwell->Whitechapel->Stepney Green
Path cost : 14
Nodes Expanded : 27
---------------------------------
Using Uniform cost Search
Shortest path : New Cross Gate->Surrey Quays->Canada Water->Rotherhithe->Wapping->Shadwell->Whitechapel->Stepney Green
Path cost : 14
Nodes Expanded : 19


In [12]:
print("Route 3 Ealing Broadway to Stepney Green")
print("Using Depth First Search")
path1, cost1, nodes1 = depth_first_search(tube_dict, 'Ealing Broadway', 'South Kensington', 0, [], 0, 30)
print_path(path1, cost1, nodes1)
print('---------------------------------')
print("Using Breadth First Search")
path2, cost2, nodes2 = breadth_first_search(tube_dict, 'Ealing Broadway', 'South Kensington')
print_path(path2, cost2, nodes2)
print('---------------------------------')
print("Using Uniform cost Search")
path3, cost3, nodes3 = uniform_cost_search(tube_dict, 'Ealing Broadway', 'South Kensington')
print_path(path3, cost3, nodes3)

Route 3 Ealing Broadway to Stepney Green
Using Depth First Search
Shortest path : Ealing Broadway->Ealing Common->North Ealing->Park Royal->Alperton->Sudbury Town->Sudbury Hill->South Harrow->Rayners Lane->West Harrow->Harrow-on-the-Hill->Northwick Park->Preston Road->Wembley Park->Finchley Road->Baker Street->Edgware Road->Paddington->Royal Oak->Westbourne Park->Ladbroke Grove->Latimer Road->Shepherd's Bush->Goldhawk Road->Hammersmith->Barons Court->Earls' Court->High Street Kensington->Gloucester Road->South Kensington
Path cost : 20
Nodes Expanded : 9
---------------------------------
Using Breadth First Search
Shortest path : Ealing Broadway->Ealing Common->Acton Town->Turnham Green->Hammersmith->Barons Court->Earls' Court->Gloucester Road->South Kensington
Path cost : 20
Nodes Expanded : 49
---------------------------------
Using Uniform cost Search
Shortest path : Ealing Broadway->Ealing Common->Acton Town->Turnham Green->Hammersmith->Barons Court->Earls' Court->Gloucester Road->

In [13]:
print("Route 4 Baker Street to Wembley Park")
print("Using Depth First Search")
path1, cost1, nodes1 = depth_first_search(tube_dict, 'Baker Street', 'Wembley Park', 0, [], 0,30)
print_path(path1, cost1, nodes1)
print('---------------------------------')
print("Using Breadth First Search")
path2, cost2, nodes2 = breadth_first_search(tube_dict, 'Baker Street', 'Wembley Park')
print_path(path2, cost2, nodes2)
print('---------------------------------')
print("Using Uniform cost Search")
path3, cost3, nodes3 = uniform_cost_search(tube_dict, 'Baker Street', 'Wembley Park')
print_path(path3, cost3, nodes3)

Route 4 Baker Street to Wembley Park
Using Depth First Search
Shortest path : Baker Street->St. John's Wood->Swiss Cottage->Finchley Road->West Hampstead->Kilburn->Willesden Green->Dollis Hill->Neasden->Wembley Park
Path cost : 20
Nodes Expanded : 9
---------------------------------
Using Breadth First Search
Shortest path : Baker Street->Finchley Road->Wembley Park
Path cost : 13
Nodes Expanded : 13
---------------------------------
Using Uniform cost Search
Shortest path : Baker Street->Finchley Road->Wembley Park
Path cost : 13
Nodes Expanded : 83


2.3 Extending the cost function
Consider how you might improve the cost function over simply adding the time required between
stations. Implement and use your new cost function in USC. 

In [14]:
def pop_frontier(frontier):
    if len(frontier) == 0:
        return None
    # copied_list = frontier.copy()
    min_val = sys.maxsize
    max_values = []
    for key, path in frontier:
        if key == min_val:
            max_values.append(path)
        elif key < min_val:
            min_val = key
            max_values.clear()
            max_values.append(path)

    max_values = sorted(max_values, key=lambda x: x[-1])
    # max_values.sort()
    desired_value = max_values[0]
    frontier.remove((min_val, max_values[0]))
    return min_val, desired_value


def get_frontier_params_new(node, frontier):
    for i in range(len(frontier)):
        curr_tuple = frontier[i]
        cost, path = curr_tuple
        if path[-1] == node:
            return True, i, cost, path

    return False, None, None, None

#find distance between zone
def euclidean_distance(x1,y1,x2,y2):
    return ((x1-x2)**2 + (y1-y2)**2 )**(0.5)

def uniform_cost_search_extended(graph, start, end):   
    """
    graph - tube dictionary
    start - start station
    end - final destination
    """
    path = list()    #array to store path data
    explored_nodes = list() #array to store explored nodes data   
    
    if start == end: #check if starting station is same as end station   
        return path, 0, len(explored_nodes) #if so return null values    
    
    path.append(start)  #add starting sation to path data
    
    path_cost = 0  
    
    frontier = [(path_cost, path)] #priority queue contianing cost and path
    
    while len(frontier) > 0: #check is queue is empty
        
        path_cost_till_now, path_till_now = pop_frontier(frontier)   #get node with min cost
        current_node = path_till_now[-1]    
        explored_nodes.append(current_node)    #append min cost node to explored node array
    
        if current_node == end:    #check if destination is reached 
            return path_till_now, path_cost_till_now, len(explored_nodes)  #return shortest path , cost and explored nodes   
    
        """
        Sorting paths based on cost 
        """
        if current_node in graph:
            nodes = []
            vals = []
            prime_zone = []
            second_zone = []
            for node, val in graph[current_node].items():
                nodes.append(node)
                vals.append(val[0])
                prime_zone.append([val[1]])
                second_zone.append([val[2]])
    
        
            sorted_node = [y for _,y in sorted(zip(vals,nodes),key=lambda x: x[0])]
            sorted_val = sorted(vals)
        
            sorted_prime_zone = [y for _,y in sorted(zip(vals,prime_zone),key=lambda x: x[0])] 
            sorted_second_zone = [y for _,y in sorted(zip(vals,second_zone),key=lambda x: x[0])]
        
            #considering start node zones as 0, 0
            init_prime_zone = '0' 
            init_second_zone = '0'
    
            for neighbour, cost, pri_zone, sec_zone in zip(sorted_node,sorted_val,sorted_prime_zone,sorted_second_zone):    
                path_to_neighbour = path_till_now.copy()    
                path_to_neighbour.append(neighbour)    
                neighbour_cost = cost + path_cost_till_now 
            
                #extending our cost function
                zone_change_cost = 0 
                if ord(init_prime_zone)<ord(pri_zone[0]): #check if there is change in primary zone 
                    zone_change_cost += 2 #if so add add 2 to cost
                
                if ord(init_second_zone)<ord(sec_zone[0]): #check if there is change in secondary zone
                    zone_change_cost += 4 #if so add 4 to cost
                
                neighbour_cost += zone_change_cost
            
                init_prime_zone = pri_zone[0]
                init_second_zone = sec_zone[0]
            
                new_element = (neighbour_cost, path_to_neighbour)    
    
                is_there, indexx, neighbour_old_cost, _ = get_frontier_params_new(neighbour, frontier)    
    
                if (neighbour not in explored_nodes) and not is_there:    #check if current neighbour not been explored
                    frontier.append(new_element)    
                
                elif is_there:    
                    #updating min cost of nodes 
                    if neighbour_old_cost > neighbour_cost:    
                        frontier.pop(indexx)    
                        frontier.append(new_element)      
    
    return [], path_cost, -1  

In [15]:
print('---------------------------------')
print("Using Extended Uniform cost Search")
path, cost, nodes = uniform_cost_search_extended(tube_dict, 'Wembley Park', 'Mile End')
print_path(path, cost, nodes)

---------------------------------
Using Extended Uniform cost Search
Shortest path : Wembley Park->Finchley Road->Baker Street->Bond Street->Green Park->Westminster->Waterloo->Bank/Monument->Liverpool Street->Bethnal Green->Mile End
Path cost : 37
Nodes Expanded : 133


2.4 Heuristic search

In [16]:
def heuristic(x1,y1, x2,y2): # Calculates the admissible heuristic of a node
    """
    x1 - node 1 primary zone
    y1 - node 1 secondary zone
    x2 - node 2 primary zone
    y2 - node 2 secondary zone
    """
    return abs(x1-x2) + abs(y1-y1) # Return calculation of admissible heuristic (manhattan distance)


In [17]:
from queue import PriorityQueue

def Astar(graph, start, end):
    
    admissible_heuristics = {} #dictionary to store node as key and node heuristic as key
    
    #conside start zones as 0,0
    start_prime_zone = '0' 
    start_second_zone  = '0'
    h = heuristic(ord(start_prime_zone),ord(start_second_zone),ord(start_prime_zone),ord(start_second_zone))
    
    admissible_heuristics[start] = 0 #updating start node heuristic as 0
    
    visited_nodes = {}  #dictionary to store visited nodes and heuristic of that node
    visited_nodes[start] = (h, [start]) 
    
    paths_to_explore = PriorityQueue() #queue to store unexplored path data
    paths_to_explore.put((h, [start], 0))
    
    while not paths_to_explore.empty(): #check if all paths are explored
        
        _, path, total_cost = paths_to_explore.get() #get a path and cost from queue and remove that element from queue
        
        current_node = path[-1] #get end element in path
        
        if current_node in graph: #check if current_element is in path 
            for neighbor, val in graph[current_node].items(): #get neighbours of current element
                
                if neighbor in admissible_heuristics: #if neighbour heuristic is calculated than get current neighbour heuristic
                    h = admissible_heuristics[neighbor] 
                else:#else calcualte heuristic 
                    #val[1][0] - primary zone of node
                    #val[2][0] - secondary zone of node
                    h = heuristic(ord(start_prime_zone),ord(start_second_zone), ord(val[1][0]),ord(val[2][0]))
                    admissible_heuristics[neighbor] = h

                #update new cost including heuristic
                new_cost = total_cost + val[0] 
                new_cost_plus_h = new_cost + h
            
                #get next node to be eplored
                if (neighbor not in visited_nodes) or (visited_nodes[neighbor][0]>new_cost_plus_h):
                    next_node = (new_cost_plus_h, path+[neighbor], new_cost)
                    visited_nodes[neighbor] = next_node 
                    paths_to_explore.put(next_node) 

    return visited_nodes[end] #return eplored node with total heuristic and path 

In [18]:
print("Using A star")
path_a = Astar(tube_dict, 'Ealing Broadway', 'South Kensington')
print('Shortest path : ',end='')
try:
    print(path_a[1][0],end='')
    for i in range(1,len(path_a[1])):
        print('->',end='')
        print(path_a[1][i],end='')
    print(f'\nPath cost : {path_a[2]}')
    print(f'Path heuristic : {path_a[0]}')
except:
    print('Path not found') 


Using A star
Shortest path : Ealing Broadway->Ealing Common->Acton Town->Turnham Green->Hammersmith->Barons Court->Earls' Court->Gloucester Road->South Kensington
Path cost : 20
Path heuristic : 21
